In [1]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
#matplotlib inline

In [11]:
data_url = 'http://hilpisch.com/tr_eikon_eod_data.csv'
raw = pd.read_csv(data_url, index_col=0,parse_dates=True)
raw.dropna(inplace=True)

raw.head()

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,.SPX,.VIX,EUR=,XAU=,GDX,GLD
Date,,,,,,,,,,,,
2010-01-04,30.572827,30.950,20.88,133.90,173.08,113.33,1132.99,20.04,1.4411,1120.00,47.71,109.80
2010-01-05,30.625684,30.960,20.87,134.69,176.14,113.63,1136.52,19.35,1.4368,1118.65,48.17,109.70
2010-01-06,30.138541,30.770,20.80,132.25,174.26,113.71,1137.14,19.16,1.4412,1138.50,49.34,111.51
2010-01-07,30.082827,30.452,20.60,130.00,177.67,114.19,1141.69,19.06,1.4318,1131.90,49.10,110.82
2010-01-08,30.282827,30.660,20.83,133.52,174.31,114.57,1144.98,18.13,1.4412,1136.10,49.84,111.37


In [5]:
symbols = ['MSFT.O']

In [6]:
data = pd.DataFrame(raw[symbols])

In [7]:
def TestRun(duration):
    
    data = pd.DataFrame(raw[symbols])
#    data['Returns'] = []
#    data['MEAN'] = []
#    data['SIGNAL'] = []
#    data['Strategy'] = []
    
    
    data['Returns'] = np.log(data[symbols] / data[symbols].shift(1))
    data.dropna(inplace=True)
    
    data['MEAN'] = data['Returns'].rolling(window=duration).mean()
    data.dropna(inplace=True)
    
    data['SIGNAL'] = np.where(data['MEAN'] > 0, 1, -1)
    
    data['Strategy'] = data['SIGNAL'].shift(1) * data['Returns']
    data.dropna(inplace=True)
    
    data['Direction'] = np.sign(data['Returns'])
    
#    hit_ratio = data['SIGNAL'].shift(1) == data['Direction']
#    print("Hit Ration was : %i" % (hit_ratio.value_counts(True)/len(hit_ratio)))
    if np.exp(data['Strategy'].sum()) > np.exp(data['Returns'].sum()):
        data[['Returns','Strategy']].cumsum().apply(np.exp).plot(figsize=(15,10))
    
    return (np.exp(data[['Returns', 'Strategy']].sum()))

In [9]:
TestRun(5)

Returns     3.257681
Strategy    0.516623
dtype: float64

In [10]:
for i in range(1,100,1):
    Returns, Strategy = TestRun(i)
    
    if Strategy > Returns:
        print("This duration wins")
        print(i)
        print("B&H\t\tAlgo")
        print(Returns,Strategy)
